### Import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

### Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### Data load & pre-processing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

### Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

### Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)
#sigmoid 대신 relu 사용
def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

### Create model 

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            #relut사용
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        #relu 사용
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

### Define data & hyper-parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x)

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

### Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### Restore checkpoint & start train or test phase

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Success to read nn_relu-468-1
 [*] Load SUCCESS
Epoch: [ 0] [    0/60000] time: 1.5726, train_loss: 2.12958097, train_accuracy: 0.3750, test_Accuracy: 0.2313
Epoch: [ 0] [    1/60000] time: 1.7749, train_loss: 2.11104727, train_accuracy: 0.3203, test_Accuracy: 0.3136
Epoch: [ 0] [    2/60000] time: 2.0442, train_loss: 2.03263474, train_accuracy: 0.5156, test_Accuracy: 0.4150
Epoch: [ 0] [    3/60000] time: 2.3205, train_loss: 1.95823193, train_accuracy: 0.5391, test_Accuracy: 0.5134
Epoch: [ 0] [    4/60000] time: 2.6147, train_loss: 1.89562249, train_accuracy: 0.5703, test_Accuracy: 0.5754
Epoch: [ 0] [    5/60000] time: 2.9358, train_loss: 1.79382968, train_accuracy: 0.5859, test_Accuracy: 0.6229
Epoch: [ 0] [    6/60000] time: 3.2799, train_loss: 1.68151391, train_accuracy: 0.6953, test_Accuracy: 0.6576
Epoch: [ 0] [    7/60000] time: 3.5643, train_loss: 1.56837761, train_accuracy: 0.7109, test_Accuracy: 0.6822
Epoch: [ 0] [    8/60000] time: 3.8893,

Epoch: [ 0] [   74/60000] time: 24.5401, train_loss: 0.36083120, train_accuracy: 0.8828, test_Accuracy: 0.9039
Epoch: [ 0] [   75/60000] time: 24.8672, train_loss: 0.33892435, train_accuracy: 0.8828, test_Accuracy: 0.9022
Epoch: [ 0] [   76/60000] time: 25.1884, train_loss: 0.24626741, train_accuracy: 0.9062, test_Accuracy: 0.9031
Epoch: [ 0] [   77/60000] time: 25.4985, train_loss: 0.33061486, train_accuracy: 0.9062, test_Accuracy: 0.9045
Epoch: [ 0] [   78/60000] time: 25.8377, train_loss: 0.21371807, train_accuracy: 0.9297, test_Accuracy: 0.9082
Epoch: [ 0] [   79/60000] time: 26.1857, train_loss: 0.32490206, train_accuracy: 0.9062, test_Accuracy: 0.9101
Epoch: [ 0] [   80/60000] time: 26.4859, train_loss: 0.42710879, train_accuracy: 0.8906, test_Accuracy: 0.9121
Epoch: [ 0] [   81/60000] time: 26.8171, train_loss: 0.29107803, train_accuracy: 0.8906, test_Accuracy: 0.9068
Epoch: [ 0] [   82/60000] time: 27.1634, train_loss: 0.38436189, train_accuracy: 0.8750, test_Accuracy: 0.9027
E

Epoch: [ 0] [  148/60000] time: 47.6125, train_loss: 0.17481034, train_accuracy: 0.9375, test_Accuracy: 0.9313
Epoch: [ 0] [  149/60000] time: 47.9127, train_loss: 0.15140563, train_accuracy: 0.9688, test_Accuracy: 0.9317
Epoch: [ 0] [  150/60000] time: 48.1604, train_loss: 0.22565380, train_accuracy: 0.9453, test_Accuracy: 0.9304
Epoch: [ 0] [  151/60000] time: 48.4493, train_loss: 0.29319537, train_accuracy: 0.9141, test_Accuracy: 0.9304
Epoch: [ 0] [  152/60000] time: 48.8013, train_loss: 0.27512455, train_accuracy: 0.9219, test_Accuracy: 0.9302
Epoch: [ 0] [  153/60000] time: 49.1404, train_loss: 0.29045966, train_accuracy: 0.9297, test_Accuracy: 0.9312
Epoch: [ 0] [  154/60000] time: 49.4595, train_loss: 0.18130413, train_accuracy: 0.9453, test_Accuracy: 0.9329
Epoch: [ 0] [  155/60000] time: 49.7977, train_loss: 0.17975520, train_accuracy: 0.9297, test_Accuracy: 0.9336
Epoch: [ 0] [  156/60000] time: 50.1248, train_loss: 0.26468277, train_accuracy: 0.9375, test_Accuracy: 0.9339
E

Epoch: [ 0] [  222/60000] time: 70.6550, train_loss: 0.25097847, train_accuracy: 0.9297, test_Accuracy: 0.9458
Epoch: [ 0] [  223/60000] time: 70.9910, train_loss: 0.24712014, train_accuracy: 0.9219, test_Accuracy: 0.9443
Epoch: [ 0] [  224/60000] time: 71.3012, train_loss: 0.15439892, train_accuracy: 0.9453, test_Accuracy: 0.9434
Epoch: [ 0] [  225/60000] time: 71.5191, train_loss: 0.26567790, train_accuracy: 0.8906, test_Accuracy: 0.9431
Epoch: [ 0] [  226/60000] time: 71.8598, train_loss: 0.17376672, train_accuracy: 0.9531, test_Accuracy: 0.9426
Epoch: [ 0] [  227/60000] time: 72.2047, train_loss: 0.14482161, train_accuracy: 0.9453, test_Accuracy: 0.9441
Epoch: [ 0] [  228/60000] time: 72.4860, train_loss: 0.14374928, train_accuracy: 0.9609, test_Accuracy: 0.9446
Epoch: [ 0] [  229/60000] time: 72.7962, train_loss: 0.30934519, train_accuracy: 0.9062, test_Accuracy: 0.9449
Epoch: [ 0] [  230/60000] time: 73.1203, train_loss: 0.16343319, train_accuracy: 0.9375, test_Accuracy: 0.9451
E

Epoch: [ 0] [  296/60000] time: 93.9959, train_loss: 0.11557761, train_accuracy: 0.9766, test_Accuracy: 0.9501
Epoch: [ 0] [  297/60000] time: 94.2859, train_loss: 0.07556620, train_accuracy: 0.9922, test_Accuracy: 0.9495
Epoch: [ 0] [  298/60000] time: 94.5991, train_loss: 0.06532626, train_accuracy: 0.9922, test_Accuracy: 0.9489
Epoch: [ 0] [  299/60000] time: 94.9172, train_loss: 0.20513059, train_accuracy: 0.9688, test_Accuracy: 0.9497
Epoch: [ 0] [  300/60000] time: 95.2264, train_loss: 0.13142064, train_accuracy: 0.9453, test_Accuracy: 0.9488
Epoch: [ 0] [  301/60000] time: 95.5206, train_loss: 0.15573893, train_accuracy: 0.9531, test_Accuracy: 0.9492
Epoch: [ 0] [  302/60000] time: 95.7730, train_loss: 0.18910357, train_accuracy: 0.9453, test_Accuracy: 0.9496
Epoch: [ 0] [  303/60000] time: 96.1111, train_loss: 0.15596296, train_accuracy: 0.9453, test_Accuracy: 0.9490
Epoch: [ 0] [  304/60000] time: 96.4432, train_loss: 0.32708818, train_accuracy: 0.9219, test_Accuracy: 0.9500
E

Epoch: [ 0] [  370/60000] time: 116.8297, train_loss: 0.05026769, train_accuracy: 0.9922, test_Accuracy: 0.9557
Epoch: [ 0] [  371/60000] time: 117.1588, train_loss: 0.08188246, train_accuracy: 0.9688, test_Accuracy: 0.9551
Epoch: [ 0] [  372/60000] time: 117.4612, train_loss: 0.22666210, train_accuracy: 0.9531, test_Accuracy: 0.9538
Epoch: [ 0] [  373/60000] time: 117.7935, train_loss: 0.12906921, train_accuracy: 0.9531, test_Accuracy: 0.9529
Epoch: [ 0] [  374/60000] time: 118.1182, train_loss: 0.10262235, train_accuracy: 0.9688, test_Accuracy: 0.9517
Epoch: [ 0] [  375/60000] time: 118.3935, train_loss: 0.15441713, train_accuracy: 0.9688, test_Accuracy: 0.9530
Epoch: [ 0] [  376/60000] time: 118.7057, train_loss: 0.28207409, train_accuracy: 0.9375, test_Accuracy: 0.9539
Epoch: [ 0] [  377/60000] time: 119.0368, train_loss: 0.22024368, train_accuracy: 0.9531, test_Accuracy: 0.9544
Epoch: [ 0] [  378/60000] time: 119.2671, train_loss: 0.13306773, train_accuracy: 0.9688, test_Accuracy:

Epoch: [ 0] [  444/60000] time: 139.2417, train_loss: 0.10973810, train_accuracy: 0.9609, test_Accuracy: 0.9543
Epoch: [ 0] [  445/60000] time: 139.4901, train_loss: 0.13128054, train_accuracy: 0.9609, test_Accuracy: 0.9530
Epoch: [ 0] [  446/60000] time: 139.8034, train_loss: 0.13477962, train_accuracy: 0.9609, test_Accuracy: 0.9528
Epoch: [ 0] [  447/60000] time: 140.1464, train_loss: 0.12740952, train_accuracy: 0.9609, test_Accuracy: 0.9519
Epoch: [ 0] [  448/60000] time: 140.4535, train_loss: 0.12257168, train_accuracy: 0.9688, test_Accuracy: 0.9528
Epoch: [ 0] [  449/60000] time: 140.7487, train_loss: 0.05466266, train_accuracy: 0.9922, test_Accuracy: 0.9538
Epoch: [ 0] [  450/60000] time: 141.0835, train_loss: 0.10212748, train_accuracy: 0.9766, test_Accuracy: 0.9542
Epoch: [ 0] [  451/60000] time: 141.4169, train_loss: 0.10876169, train_accuracy: 0.9609, test_Accuracy: 0.9555
Epoch: [ 0] [  452/60000] time: 141.7501, train_loss: 0.13108101, train_accuracy: 0.9609, test_Accuracy:

### Accuracy : 96.22%